In [1]:
import lxml.html
import lxml.etree
import requests
import pandas as pd
import numpy as np

In [54]:
start_link = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'
response = requests.get(start_link)
tree = lxml.html.fromstring(response.text)
region_links = tree.xpath('//option/@value')

df = pd.DataFrame()
columns = []
for reg_link in region_links:
    reg_tree = lxml.html.fromstring(requests.get(reg_link).text)
    if not columns:
        columns = reg_tree.xpath('//table/tr/td[contains(@align,"left")]/nobr/text()')
    tik_links = reg_tree.xpath('//option/@value')
    region = reg_tree.xpath('///table/tr/td/a/text()')[1]
    print(region, ' started')
    for tik_link in tik_links:
        
        tik_tree = lxml.html.fromstring(requests.get(tik_link).text)
        tik_name = tik_tree.xpath('//table/tr/td/a/text()')[2]
        
        uik_link = tik_tree.xpath('//table//@href')[3]
        uik_tree = lxml.html.fromstring(requests.get(uik_link).text)
        
        data = np.array(uik_tree.xpath('//table/tr/td[contains(@align,"right")]/nobr/b/text()')[20:])
        data = data.reshape(20, -1).transpose()
        
        uik_numbers = uik_tree.xpath('//table/tr[contains(@valign, "top")]/td[contains(@align,"center")]/nobr/text()')
        
        dtemp = pd.DataFrame(data, columns=columns)
        dtemp['название региона'] = region
        dtemp['название ТИК'] = tik_name
        dtemp['номер УИК'] = uik_numbers
        
        df = df.append(dtemp, ignore_index=True)
        

Республика Адыгея (Адыгея)  started
Республика Алтай  started
Республика Башкортостан  started
Республика Бурятия  started
Республика Дагестан  started
Республика Ингушетия  started
Кабардино-Балкарская Республика  started
Республика Калмыкия  started
Карачаево-Черкесская Республика  started
Республика Карелия  started
Республика Коми  started
Республика Крым  started
Республика Марий Эл  started
Республика Мордовия  started
Республика Саха (Якутия)  started
Республика Северная Осетия - Алания  started
Республика Татарстан (Татарстан)  started
Республика Тыва  started
Удмуртская Республика  started
Республика Хакасия  started
Чеченская Республика  started
Чувашская Республика - Чувашия  started
Алтайский край  started
Забайкальский край  started
Камчатский край  started
Краснодарский край  started
Красноярский край  started
Пермский край  started
Приморский край  started
Ставропольский край  started
Хабаровский край  started
Амурская область  started
Архангельская область  started
Астр

In [55]:
df.sample(10)

,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,Число бюллетеней в стационарных ящиках для голосования,Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,...,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич,название региона,название ТИК,номер УИК
10250,856,900,0,469,140,291,140,469,3,606,...,114,29,440,6,5,8,2,Республика Марий Эл,Медведевская,УИК №344
61096,1342,1300,0,822,26,452,26,822,13,835,...,114,43,598,33,8,12,15,Нижегородская область,"Дзержинская, Калининская",УИК №928
54795,1650,1650,0,899,9,742,9,899,21,887,...,114,56,652,19,5,9,22,Московская область,Истринская городская,УИК №750
87010,315,320,0,219,18,83,18,219,1,236,...,17,24,192,0,0,1,0,Ульяновская область,Чердаклинская,УИК №3110
95744,2051,2000,0,1407,45,548,45,1407,13,1439,...,123,27,1168,53,6,12,42,город Санкт-Петербург,Территориальная избирательная комиссия №28,УИК №1919
37492,951,900,0,548,62,290,62,548,5,605,...,84,66,439,3,4,1,1,Владимирская область,Вязниковская,УИК №782
15120,310,345,0,268,39,38,39,268,6,301,...,3,5,290,1,2,0,0,Республика Татарстан (Татарстан),Тетюшская,УИК №2608
45805,2039,1500,0,1108,35,357,35,1108,11,1132,...,120,65,867,41,7,12,15,Калужская область,"Калуга, Ленинская",УИК №908
65514,440,440,0,231,42,167,42,231,2,271,...,75,8,177,4,0,1,2,Оренбургская область,Абдулинская городская,УИК №32
58641,21,21,0,21,0,0,0,21,0,21,...,1,4,15,0,0,0,0,Мурманская область,Мурманская,УИК №557


In [56]:
df.to_csv('results.csv')